<h1>Table of contents</h1>

<div class="alert alert-block alert-info"
     style="margin-top: 20px">
    <ol>
        <li><a href="#Introduction">Introduction</a></li>
        <li>
            <a href="#Data_Description">Data Description</a>
            <ol>
                <li><a href="#Imports">Import necessary Libraries</a></li>
                <li><a href="#FourSquare">Four Square API Setup</a></li>
                <li><a href="#Coordinates">Converting User Provided Address to Geographical Coordinates</a></li>
                <li><a href="#VenueData">Get Venues from Four Square API</a></li>
                <li><a href="#Repeat">Repeat process for second location</a></li>
            </ol>
        </li>
        <li><a href="#Methodology">Methodology</a></li>
        <li><a href="#EDA">Exploratory Data Analysis</a></li>
        <li><a href="#Results">Results</a></li>
        <li><a href="#Discussion">Discussion</a></li>
        <li><a href="#Conclusion">Conclusion</a></li>
    </ol>
</div>

<div id="Introduction">
    <h1>Introduction</h1>
</div>
<hr>
<h2>
    Description & Discussion of the Background
</h2>
<p>
Having come from the NJ/NY Metropolitan area, I was use to having about 20 - 30 restaurants within earshot. I was a bit surprised that this was not the case in Colorado Springs. So, in order to help others avoid being in a similar situation, I want to develop an application that compares the venue density of two different locations provided by the user in order to determine the similarity of the locations in terms of the amount of restaurants nearby. So they can determine where to go travel to with this in mind
</p>

<div id="Data_Description">
    <h2>
        Data Description
    </h2>
</div>
<hr>
<p>
    This problem only requires the location data provided from the user. The application will be leveraging packages like geopy that can be used to translate an address provided by the user into latitude and longitude coordinates that can be passed into the Four Square API. Then the API will return the venure, likes, tip information for that location. Since this project is related to density it will rely on the amount of venues located in the area. Beyond the venues of the two locations, there is not much Data Collection required for this application beyond what is provided by the Four Square API.
</p>

<div id="Imports">
    <h3>A. Import necessary Libraries</h3>
</div>

In [1]:
import requests     # library to handle requests
import pandas as pd # library to handle data analysis
import numpy as np  # library to handle data like vectors
import random       # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # turns an address into (lat, long)

# transforming json into a pandas dataframe
from pandas.io.json import json_normalize

!pip install folium==0.5.0
import folium # Plotting library

print('Folium installed')
print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Folium installed
Libraries imported.


<div id="FourSquare">
    <h2>B. Four Square API Setup</h2>
</div>
Need to provide Four Square API with the Credentials it needs in order to use it. This requires the user to have at least a developer account with FourSquare in order to use the API that they provide allowing for the creation of the visualizations through Folium. Here is a link to setup an account with <a href="https://foursquare.com/developers/signup">FourSquare</a>

In [73]:
CLIENT_ID = '***' # your Foursquare ID
CLIENT_SECRET = '***' # your Foursquare Secret
ACCESS_TOKEN = '***' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ***
CLIENT_SECRET:***


<div id="Coordinates">
    <h2>
        C. Converting User Provided Address to Geographical Coordinates
    </h2>
</div>

First, the user provides their adddress to the program, and we use the geopy library's Nominatim calss to convert the address provided into a geographical coordinate that can be used by Folium. But in order to use Nominatim, we need to define an instance of the geocoder and an user_agent. We will name our agent fsq_agent. Then we will print the results of the latitude and longitude

In [18]:
address = '150 Greenwich Street, Manhattan, NY'

geolocator = Nominatim(user_agent='fsq_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.710318650000005 -74.01196856508565


In [19]:
search_query = 'Food'
radius = 1200
print(search_query + '...OK!')

Food...OK!


<b>Define the corresponding URL</b>

In [74]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

<div id="VenueData">
    <h2>
        D. Get Venues from Four Square API
    </h2>
</div>

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60d2b1c639d25c500a9cb6dd'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '49bc3b0af964a52020541fe3',
    'name': 'Whole Foods Market',
    'location': {'address': '270 Greenwich St',
     'crossStreet': 'at Warren St',
     'lat': 40.715579155420606,
     'lng': -74.01136823958119,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.715579155420606,
       'lng': -74.01136823958119}],
     'distance': 587,
     'postalCode': '10007',
     'cc': 'US',
     'neighborhood': 'Tribeca',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['270 Greenwich St (at Warren St)',
      'New York, NY 10007']},
    'categories': [{'id': '4bf58dd8d48988d118951735',
      'name': 'Grocery Store',
      'pluralName': 'Grocery Stores',
      'shortName': 'Grocery Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/fo

In [22]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# transform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,49bc3b0af964a52020541fe3,Whole Foods Market,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",v-1624420806,False,270 Greenwich St,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",...,New York,NY,United States,"[270 Greenwich St (at Warren St), New York, NY...",NaN,NaN,NaN,NaN,NaN,NaN
1,4bbb7d5aafe1b713ac3f2f4b,Food Bank For New York City,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1624420806,False,39 Broadway Fl 10,NaN,40.706336,-74.012972,"[{'label': 'display', 'lat': 40.70633552386926...",...,New York,NY,United States,"[39 Broadway Fl 10, New York, NY 10006]",NaN,NaN,NaN,NaN,NaN,NaN
2,4bba18d11261d13adc36ea98,Adel's best #1 Halal Food Cart,"[{'id': '4bf58dd8d48988d10b941735', 'name': 'F...",v-1624420806,False,Maiden Lane & Front street,Maiden Lane & Front street,40.705609,-74.005599,"[{'label': 'display', 'lat': 40.70560923100874...",...,New York,NY,United States,[Maiden Lane & Front street (Maiden Lane & Fro...,NaN,NaN,NaN,NaN,NaN,NaN
3,4b969958f964a5209ed734e3,Halal Fast Food (cart),"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1624420806,False,Greenwich and Murray,Barclay Street,40.715147,-74.011504,"[{'label': 'display', 'lat': 40.71514746067547...",...,New York,NY,United States,"[Greenwich and Murray (Barclay Street), New Yo...",NaN,NaN,NaN,NaN,NaN,NaN
4,56e1be06498ef45e2b30a076,Liberty Halal Food,"[{'id': '56aa371be4b08b9a8d57350b', 'name': 'F...",v-1624420806,False,Water Street,Whitehall Street,40.702343,-74.012700,"[{'label': 'display', 'lat': 40.702343, 'lng':...",...,New York,NY,United States,"[Water Street (Whitehall Street), New York, NY]",NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,Whole Foods Market,Grocery Store,270 Greenwich St,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",587,10007,US,Tribeca,New York,NY,United States,"[270 Greenwich St (at Warren St), New York, NY...",49bc3b0af964a52020541fe3
1,Food Bank For New York City,Office,39 Broadway Fl 10,NaN,40.706336,-74.012972,"[{'label': 'display', 'lat': 40.70633552386926...",451,10006,US,NaN,New York,NY,United States,"[39 Broadway Fl 10, New York, NY 10006]",4bbb7d5aafe1b713ac3f2f4b
2,Adel's best #1 Halal Food Cart,Falafel Restaurant,Maiden Lane & Front street,Maiden Lane & Front street,40.705609,-74.005599,"[{'label': 'display', 'lat': 40.70560923100874...",750,10038,US,NaN,New York,NY,United States,[Maiden Lane & Front street (Maiden Lane & Fro...,4bba18d11261d13adc36ea98
3,Halal Fast Food (cart),Food Truck,Greenwich and Murray,Barclay Street,40.715147,-74.011504,"[{'label': 'display', 'lat': 40.71514746067547...",538,10007,US,NaN,New York,NY,United States,"[Greenwich and Murray (Barclay Street), New Yo...",4b969958f964a5209ed734e3
4,Liberty Halal Food,Food Stand,Water Street,Whitehall Street,40.702343,-74.012700,"[{'label': 'display', 'lat': 40.702343, 'lng':...",889,NaN,US,NaN,New York,NY,United States,"[Water Street (Whitehall Street), New York, NY]",56e1be06498ef45e2b30a076
5,Biryani House Food Truck,Food Truck,140 Broadway,Beaver St Btw Broadway & New St,40.704956,-74.012537,"[{'label': 'display', 'lat': 40.70495624253704...",598,NaN,US,Financial District,NaN,New York,United States,[140 Broadway (Beaver St Btw Broadway & New St...,4ded108b52b100c01e04d28b
6,Halal Food Truck,Food Truck,Wall Street & Water Street,NaN,40.705002,-74.008021,"[{'label': 'display', 'lat': 40.70500214280706...",679,10005,US,NaN,New York,NY,United States,"[Wall Street & Water Street, New York, NY 10005]",4cea0c3ef8653704eebebac4
7,Turnstile Tours Food Cart Tour - Financial Dis...,General Entertainment,Wall St,Water St,40.705042,-74.007436,"[{'label': 'display', 'lat': 40.7050423443854,...",700,10005,US,NaN,New York,NY,United States,"[Wall St (Water St), New York, NY 10005]",4be1d7540f299c749ed47ebf
8,Breakfast Food Cart,Food Truck,Water Street,Pine Street,40.707063,-74.005083,"[{'label': 'display', 'lat': 40.70706288490862...",684,10038,US,NaN,New York,NY,United States,"[Water Street (Pine Street), New York, NY 10038]",4d90a7553bfef04defe0c786
9,88 Natural Food,Deli / Bodega,88 Mulberry St,Btw Canal & Bayard Streets,40.716612,-73.998593,"[{'label': 'display', 'lat': 40.716612, 'lng':...",1328,10013,US,NaN,New York,NY,United States,"[88 Mulberry St (Btw Canal & Bayard Streets), ...",4b5b3e4df964a520c9ed28e3


In [24]:
dataframe_filtered.name

0                                    Whole Foods Market
1                           Food Bank For New York City
2                        Adel's best #1 Halal Food Cart
3                                Halal Fast Food (cart)
4                                    Liberty Halal Food
5                              Biryani House Food Truck
6                                      Halal Food Truck
7     Turnstile Tours Food Cart Tour - Financial Dis...
8                                   Breakfast Food Cart
9                                       88 Natural Food
10                              Taco Morelos Food Truck
11                                  Schnitzi Food Truck
12                        Xpress Energy Halal Food Cart
13                                   Food Sing 88 Corp.
14                                       Wang Food Cart
15                   Food Carts on Broadway and Liberty
16                                           Halal Food
17                            The Best of New Yo

In [38]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=14)

folium.CircleMarker(
    [latitude, longitude],
    radius = 10,
    color='red',
    popup='Home, Sweet Home',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(venues_map)

# add therestaura nts as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

In [75]:
venue_id = '4bba18d11261d13adc36ea98' # ID of Whole Foods Market
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

In [31]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4bba18d11261d13adc36ea98',
 'name': "Adel's best #1 Halal Food Cart",
 'contact': {},
 'location': {'address': 'Maiden Lane & Front street',
  'crossStreet': 'Maiden Lane & Front street',
  'lat': 40.70560923100874,
  'lng': -74.00559949094632,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.70560923100874,
    'lng': -74.00559949094632}],
  'postalCode': '10038',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['Maiden Lane & Front street (Maiden Lane & Front street)',
   'New York, NY 10038']},
 'canonicalUrl': 'https://foursquare.com/v/adels-best-1-halal-food-cart/4bba18d11261d13adc36ea98',
 'categories': [{'id': '4bf58dd8d48988d10b941735',
   'name': 'Falafel Restaurant',
   'pluralName': 'Falafel Restaurants',
   'shortName': 'Falafel',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/falafel_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 18},
 'pr

In [32]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.5


In [33]:
venue_id = '4bba18d11261d13adc36ea98' # ID of Whole Foods Market
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.5


In [34]:
result['response']['venue']['tips']['count']

18

In [35]:
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60d2b249664ecb59609fdfcc'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'tips': {'count': 18,
   'items': [{'id': '5d8e621db530ec0006bece40',
     'createdAt': 1569612317,
     'text': 'I’m not a falafel person but I actually really enjoy eating the falafels from this halal cart. The lines here are always long but move quickly. The food prices went up $1 so items start at $6.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5d8e621db530ec0006bece40',
     'likes': {'count': 0, 'groups': []},
     'like': False,
     'logView': True,
     'agreeCount': 0,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '13708620',
      'firstName': 'Kimmie',
      'lastName': 'Ohh',
      'gender': 'female',
      'countryCode': 'US',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/13708620_Ack0oGof_0IVzllt83nG4jMC7bEmP9XqHPfVbiEsVAOwaUtX

In [36]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'likes', 'like', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

<div id="Repeat">
    <h2>
        Repeat process for second location
    </h2>
</div>
In this step we want to convert the second address provided by the user into a geographical coordinates to be used by geopy and folium. We also want to call the Four Square API for the new location as well before comparing the density of the two addresses to one another

In [41]:
address2 = '14 E, W Cache La Poudre St, Colorado Springs, CO'

location2 = geolocator.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print(latitude2, longitude2)

38.846510603080674 -104.82468087034574


In [76]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2,ACCESS_TOKEN, VERSION, search_query, radius*2, LIMIT)

In [53]:
results2 = requests.get(url2).json()
results

{'meta': {'code': 200, 'requestId': '60d2b249664ecb59609fdfcc'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'tips': {'count': 18,
   'items': [{'id': '5d8e621db530ec0006bece40',
     'createdAt': 1569612317,
     'text': 'I’m not a falafel person but I actually really enjoy eating the falafels from this halal cart. The lines here are always long but move quickly. The food prices went up $1 so items start at $6.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5d8e621db530ec0006bece40',
     'likes': {'count': 0, 'groups': []},
     'like': False,
     'logView': True,
     'agreeCount': 0,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '13708620',
      'firstName': 'Kimmie',
      'lastName': 'Ohh',
      'gender': 'female',
      'countryCode': 'US',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/13708620_Ack0oGof_0IVzllt83nG4jMC7bEmP9XqHPfVbiEsVAOwaUtX

In [54]:
# assign relevant part of JSON to venues
venues2 = results2['response']['venues']

# transform venues into a dataframe
dataframe2 = json_normalize(venues2)
dataframe2.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.state,location.country,location.formattedAddress,location.crossStreet,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,55a56b40498e4d47f4747e99,PigLatin Food Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1624425649,False,112 E Boulder St,38.839733,-104.822647,"[{'label': 'display', 'lat': 38.839733, 'lng':...",774,...,CO,United States,"[112 E Boulder St, Colorado Springs, CO 80903]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5196c62a498e15569a006bc8,High Grade Catering And Food Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1624425649,False,Platte Ave,38.837864,-104.820930,"[{'label': 'display', 'lat': 38.83786392211914...",1015,...,CO,United States,"[Platte Ave (Nevada), Colorado Springs, CO]",Nevada,NaN,NaN,NaN,NaN,NaN,NaN
2,5cc49bba031320002c016a18,Hafa Adai Fiesta Food,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1624425649,False,109 E Pikes Peak Ave,38.833538,-104.822800,"[{'label': 'display', 'lat': 38.83353805541992...",1453,...,CO,United States,"[109 E Pikes Peak Ave, Colorado Springs, CO 80...",NaN,1720883,https://www.grubhub.com/restaurant/hafa-adai-f...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png
3,4f32786719836c91c7daf2e8,Paolas Mexican Food,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1624425649,False,26 S Wahsatch Ave,38.832684,-104.818436,"[{'label': 'display', 'lat': 38.83268356323242...",1631,...,CO,United States,"[26 S Wahsatch Ave, Colorado Springs, CO 80903]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5dbfb89d221946000899f102,Wag N' Wash Natural Pet Food & Grooming,"[{'id': '5032897c91d4c4b30a586d69', 'name': 'P...",v-1624425649,False,1625 West Uintah Street,38.847762,-104.844034,"[{'label': 'display', 'lat': 38.8477616, 'lng'...",1683,...,CO,United States,"[1625 West Uintah Street, Colorado Springs, CO...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns2 = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = dataframe2.loc[:, filtered_columns2]

# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]

dataframe_filtered2.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,PigLatin Food Truck,Food Truck,112 E Boulder St,38.839733,-104.822647,"[{'label': 'display', 'lat': 38.839733, 'lng':...",774,80903,US,Colorado Springs,CO,United States,"[112 E Boulder St, Colorado Springs, CO 80903]",NaN,55a56b40498e4d47f4747e99
1,High Grade Catering And Food Truck,Food Truck,Platte Ave,38.837864,-104.820930,"[{'label': 'display', 'lat': 38.83786392211914...",1015,NaN,US,Colorado Springs,CO,United States,"[Platte Ave (Nevada), Colorado Springs, CO]",Nevada,5196c62a498e15569a006bc8
2,Hafa Adai Fiesta Food,Restaurant,109 E Pikes Peak Ave,38.833538,-104.822800,"[{'label': 'display', 'lat': 38.83353805541992...",1453,80903,US,Colorado Springs,CO,United States,"[109 E Pikes Peak Ave, Colorado Springs, CO 80...",NaN,5cc49bba031320002c016a18
3,Paolas Mexican Food,Food,26 S Wahsatch Ave,38.832684,-104.818436,"[{'label': 'display', 'lat': 38.83268356323242...",1631,80903,US,Colorado Springs,CO,United States,"[26 S Wahsatch Ave, Colorado Springs, CO 80903]",NaN,4f32786719836c91c7daf2e8
4,Wag N' Wash Natural Pet Food & Grooming,Pet Service,1625 West Uintah Street,38.847762,-104.844034,"[{'label': 'display', 'lat': 38.8477616, 'lng'...",1683,80904,US,Colorado Springs,CO,United States,"[1625 West Uintah Street, Colorado Springs, CO...",NaN,5dbfb89d221946000899f102


In [57]:
dataframe_filtered2.name

0                        PigLatin Food Truck
1         High Grade Catering And Food Truck
2                      Hafa Adai Fiesta Food
3                        Paolas Mexican Food
4    Wag N' Wash Natural Pet Food & Grooming
5          Golden Gyros Authentic Greek Food
6                         Gather Food Studio
7                        Roc & Ro Food Truck
8                         Ranch Foods Direct
9                Mountain Mama Natural Foods
Name: name, dtype: object

In [58]:
venues_map2 = folium.Map(location=[latitude2, longitude2], zoom_start=14)

folium.CircleMarker(
    [latitude2, longitude2],
    radius = 10,
    color='red',
    popup='Away from Home',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(venues_map2)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map2)

In [66]:
num_ny_venues = dataframe_filtered.shape[0]  #Number of Venues in Greenwich 
num_co_venues = dataframe_filtered2.shape[0] #Number of Venues in Colorado College

In [67]:
ny_radius = radius
co_radius = radius * 2

In [68]:
ny_venue_density = num_ny_venues / ny_radius
co_venue_density = num_co_venues / co_radius

In [69]:
ny_venue_density

0.025

In [70]:
co_venue_density

0.004166666666666667

In [71]:
ny_venue_density > co_venue_density

True

In [72]:
ny_venue_density / co_venue_density

6.0

<div id="Methodology">
    <h1>
        Methodology
    </h1>
</div>
<hr>
After setting up the Four Square API and the Geocoder from geopy. Upon returning the data from the API, it was transformed into a DataFrame using the normalize_json function. Followed filtering out the columns that did not include locations, and putting the name and cateory of the venue in the forefront. After filtering on the columns  of the two locations, it turns out the data was ready to be used for visualization and computation. So it was really straightforward


<div id="EDA">
    <h2>
        Exploratory Data Analysis
    </h2>
</div>
<hr>
Provided visualizations for both of the addresses provided by the users via geopy, folium

<div id="Visualizations">
    <h3>
        Visualizations
    </h3>
</div>

Figure of the Manhatthan Area that I typically would get food from. The map was visualized using the FourSquare data that was converted into a data frame, filtered and used to plot the point using geopy and folium for the marker and map visualization

In [39]:
venues_map

In [59]:
venues_map2

# Results
---
So the Application was successfully created. After cleaning the data from the addresses provided by the user and converting them into the geographical coordinates. After cleaning and filtering the unused parts of the data like tips, likes and etc. The application is able to calculate the density of address1 (World Trade Center Manhattan NY) and address2 (Colorado College in Colorado Springs, CO). It was able to successfully determine that the NY/NJ area that I frequented for food is more venue dense than at my school. In fact the venue density back at home was exactly 6 times more dense than at Colorado College.



# Discussion
---
<h3> Observations made during the project</h3>
    <p>I noticed in the beginning of this project that based on the idea proposed of using venure density was going to be really simplistic. But as a consequence, beyond the comparison of the density of the two locations there is not alot of interesting things going on. Beyond the notable difference in the areas. This program is bias to urban / small town areas where maybe the community does not use four square (i.e. Newark, NJ). So I ended up having to include my travels to New York to provide results. </p>
<h3>Recommendations and Extensions / Repurposing of the project</h3>
    <p>Since the program was so simplistic it did not really need to leverage anything models, but if I had more time, I would try to see if I could label areas that are food deserts across the US. I think it would also be worthwhile to try to see how the results change based on the category of the venues (i.e. Italian, Indian, Chinese, etc). I also regret adding the extra data preprocessing of the API data using tips and counts because it was not leveraged anywhere and was actually a waste of time</p>

# Conclusion
---
## Never do you have to go in blind when it comes to traveling. I wish this application existed before I left for Colorado because I really have been spoiled by being home. Looking back on it now, I probably would have ended up somewhere entirely different if I knew that I would be in an area where the venue density was severely lower. I think this result was expected though due to going from a Metropolitan Area to a small-ish town. But this can also be used to at least avoid making the same mistake I made all those years ago and it can help you set up for your next round of travel.